In [1]:
import torch
import torch.nn as nn 
import torch.optim as optim 
from torch.utils.data import DataLoader, random_split
from data_manager import load_data, get_data_shapes
%load_ext autoreload
%autoreload 2

In [2]:
X, y, num_games = load_data(limit=800000)
print("Ilość ewaluacji pozycji:", num_games)

Przetwarzanie pozycji:   0%|          | 0/800000 [00:00<?, ?it/s]

Ilość ewaluacji pozycji: 800000


In [3]:
get_data_shapes(X,y)

{'X_shape': (800000, 8, 8, 18),
 'y_shape': (800000,),
 'X_dtype': dtype('float32'),
 'y_dtype': dtype('float32'),
 'X_min': np.float32(0.0),
 'X_max': np.float32(1.0),
 'y_min': np.float32(-1.0),
 'y_max': np.float32(1.0),
 'y_unique': 2001}

In [4]:
print(y[:2])

[ 0.   -0.01]


In [5]:
from dataset import ChessDataset
from model import SmallResNetValue as Model

dataset = ChessDataset(X, y)

train_size = int(0.8 * len(dataset))

val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

torch.manual_seed(42)

model = Model().to(device)
model

Using device: cuda


SmallResNetValue(
  (conv1): Conv2d(18, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc1): Linear(in_features=256, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=1, bias=True)
  (relu): ReLU(inplace=True)
  (tanh): Tanh()
)

In [6]:
import csv

optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.MSELoss() # ew. nn.L1Loss() lub nn.SmoothL1Loss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.5, patience=5
)

num_epochs = 100
model_name = "cn2_bn2_rlrop"
model_type = "Default + 1x Conv2d(128,256) 3x BatchNorm2d(64) and 128, ReduceLROnPlateau"

description = f"Epochs: {num_epochs}, Lr:{optimizer.param_groups[0]['lr']}, Model: {model_type}"

with open(f"../../charts/value_network/{model_name}.csv", "w", newline='') as csvfile:
    csvfile.write(f"# {description}\n")
    writer = csv.writer(csvfile)
    writer.writerow(["epoch", "train_loss", "val_loss"])

    for epoch in range(num_epochs):
        # ----- Trening -----
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            running_loss += loss.item()
        avg_train_loss = running_loss / len(train_loader)

        # ----- Walidacja -----
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        avg_val_loss = val_loss / len(val_loader)

        # ----- Scheduler -----
        scheduler.step(avg_val_loss)

        # ----- Logi -----
        writer.writerow([epoch + 1, avg_train_loss, avg_val_loss])
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

Epoch 1/100 | Train Loss: 0.0731 | Val Loss: 0.0690
Epoch 2/100 | Train Loss: 0.0601 | Val Loss: 0.0585
Epoch 3/100 | Train Loss: 0.0528 | Val Loss: 0.0497
Epoch 4/100 | Train Loss: 0.0476 | Val Loss: 0.0552
Epoch 5/100 | Train Loss: 0.0432 | Val Loss: 0.0456
Epoch 6/100 | Train Loss: 0.0395 | Val Loss: 0.0462
Epoch 7/100 | Train Loss: 0.0363 | Val Loss: 0.0410
Epoch 8/100 | Train Loss: 0.0334 | Val Loss: 0.0403
Epoch 9/100 | Train Loss: 0.0309 | Val Loss: 0.0477
Epoch 10/100 | Train Loss: 0.0285 | Val Loss: 0.0408
Epoch 11/100 | Train Loss: 0.0267 | Val Loss: 0.0390
Epoch 12/100 | Train Loss: 0.0249 | Val Loss: 0.0383
Epoch 13/100 | Train Loss: 0.0233 | Val Loss: 0.0331
Epoch 14/100 | Train Loss: 0.0219 | Val Loss: 0.0314
Epoch 15/100 | Train Loss: 0.0208 | Val Loss: 0.0316
Epoch 16/100 | Train Loss: 0.0197 | Val Loss: 0.0348
Epoch 17/100 | Train Loss: 0.0187 | Val Loss: 0.0323
Epoch 18/100 | Train Loss: 0.0178 | Val Loss: 0.0617
Epoch 19/100 | Train Loss: 0.0170 | Val Loss: 0.0470
Ep

In [7]:
torch.save(model.state_dict(), "../../models/value_network/{}.pth".format(model_name.upper()))